# Colab 2
## Frequent Pattern Mining in Spark

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
import os
os.environ["JAVA_HOME"] = "D:/java/jdk8"

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [4]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [5]:
spark

### Your task

If you run successfully the setup stage, you are ready to work with the **3 Million Instacart Orders** dataset. In case you want to read more about it, check the [official Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) about it, a concise [schema description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) of the dataset, and the [download page](https://www.instacart.com/datasets/grocery-shopping-2017).

In this Colab, we will be working only with a small training dataset (~131K orders) to perform fast Frequent Pattern Mining with the FP-Growth algorithm.

In [6]:
products = spark.read.csv('products.csv', header=True, inferSchema=True)
orders = spark.read.csv('order_products__train.csv', header=True, inferSchema=True)

In [7]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)



In [8]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)



Use the Spark Dataframe API to join 'products' and 'orders', so that you will be able to see the product names in each transaction (and not only their ids).  Then, group by the orders by 'order_id' to obtain one row per basket (i.e., set of products purchased together by one customer). 

In [9]:
# YOUR CODE HERE
joined_orders = orders.join(products,products.product_id == orders.product_id)

In [10]:
from pyspark.sql.functions import collect_list
baskets = joined_orders.groupBy("order_id").agg(collect_list("product_name").alias("product_names"))

In this Colab we will explore [MLlib](https://spark.apache.org/mllib/), Apache Spark's scalable machine learning library. Specifically, you can use its implementation of the [FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth) algorithm to perform efficiently Frequent Pattern Mining in Spark.
Use the Python example in the documentation, and train a model with 

```minSupport=0.01``` and ```minConfidence=0.5```



In [11]:
# YOUR CODE HERE
baskets.head(3)

[Row(order_id=1342, product_names=['Bag of Organic Bananas', 'Seedless Cucumbers', 'Organic Mandarins', 'Organic Strawberries', 'Versatile Stain Remover', 'Pink Lady Apples', 'Chicken Apple Sausage', 'Raw Shrimp']),
 Row(order_id=1591, product_names=['Cracked Wheat', 'Organic Greek Whole Milk Blended Vanilla Bean Yogurt', 'Navel Oranges', 'Spinach', 'Original Patties (100965) 12 Oz Breakfast', 'Cinnamon Multigrain Cereal', 'Chewy 25% Low Sugar Chocolate Chip Granola', 'Uncured Genoa Salami', 'Natural Vanilla Ice Cream', 'Lemon Yogurt', 'Strawberry Rhubarb Yoghurt', 'Garlic', 'Pure Vanilla Extract', 'Lower Sugar Instant Oatmeal  Variety', 'Organic Bunny Fruit Snacks Berry Patch', 'Buttermilk Waffles', 'Granny Smith Apples', 'Medium Scarlet Raspberries', 'Banana', 'Strawberry Banana Smoothie', 'Green Machine Juice Smoothie', 'Ultra Thin Sliced Provolone Cheese', 'Oven Roasted Turkey Breast', 'Original Turkey Burgers Smoke Flavor Added', 'Original Whole Grain Chips', 'Goldfish Pretzel Bak

Compute how many frequent itemsets and association rules were generated by running FP-growth.


In [12]:
# YOUR CODE HERE
from pyspark.ml.fpm import FPGrowth
fp_growth = FPGrowth(itemsCol="product_names",minSupport=0.01,minConfidence=0.5)
model = fp_growth.fit(baskets)

In [14]:
# YOUR CODE HERE
model.freqItemsets.count()

120

Now retrain the FP-growth model changing only 
```minsupport=0.001``` 
and compute how many frequent itemsets and association rules were generated.


In [15]:
# YOUR CODE HERE
model.associationRules.count()

0

In [26]:
# YOUR CODE HERE
fp_growth = FPGrowth(itemsCol="product_names",minSupport=0.001,minConfidence=0.5)
model = fp_growth.fit(baskets)

In [27]:
# YOUR CODE HERE
model.freqItemsets.count()

4444

In [28]:
model.associationRules.count()

11

In [21]:
# Show association rules
# YOUR CODE HERE
from pyspark.sql.functions import desc
sorted_freItemsets = model.freqItemsets.orderBy(desc("freq"))
sorted_freItemsets.show(truncate=False)

+------------------------+-----+
|items                   |freq |
+------------------------+-----+
|[Banana]                |18726|
|[Bag of Organic Bananas]|15480|
|[Organic Strawberries]  |10894|
|[Organic Baby Spinach]  |9784 |
|[Large Lemon]           |8135 |
|[Organic Avocado]       |7409 |
|[Organic Hass Avocado]  |7293 |
|[Strawberries]          |6494 |
|[Limes]                 |6033 |
|[Organic Raspberries]   |5546 |
|[Organic Blueberries]   |4966 |
|[Organic Whole Milk]    |4908 |
|[Organic Cucumber]      |4613 |
|[Organic Zucchini]      |4589 |
|[Organic Yellow Onion]  |4290 |
|[Organic Garlic]        |4158 |
|[Seedless Red Grapes]   |4059 |
|[Asparagus]             |3868 |
|[Organic Grape Tomatoes]|3823 |
|[Organic Red Onion]     |3818 |
+------------------------+-----+
only showing top 20 rows



In [22]:
model.associationRules.show(truncate=False)

+-----------------------------------------------------------------+------------------------+------------------+------------------+
|antecedent                                                       |consequent              |confidence        |lift              |
+-----------------------------------------------------------------+------------------------+------------------+------------------+
|[Organic Kiwi, Organic Hass Avocado]                             |[Bag of Organic Bananas]|0.5459770114942529|4.627719489738336 |
|[Organic Raspberries, Organic Hass Avocado, Organic Strawberries]|[Bag of Organic Bananas]|0.5984251968503937|5.072272070642333 |
|[Organic Broccoli, Organic Hass Avocado]                         |[Bag of Organic Bananas]|0.5048231511254019|4.278897986822536 |
|[Organic Unsweetened Almond Milk, Organic Hass Avocado]          |[Bag of Organic Bananas]|0.5141065830721003|4.357584667849303 |
|[Yellow Onions, Strawberries]                                    |[Banana]        